In [66]:
import time
import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from urllib.parse import quote

In [71]:
url = f'https://kin.naver.com/search/list.naver?query={quote("설 선물")}&page=1'
driver=webdriver.Chrome() #크롬드라이버 연결
driver.get(url) #url 가져오기
soup=BeautifulSoup(driver.page_source,'html.parser') #첫페이지를 soup으로

In [72]:
lis = soup.select('.basic1 > li')
len(lis)

10

In [73]:
li = lis[2]
sub_url = li.select_one('a._searchListTitleAnchor')['href']
sub_url

'https://kin.naver.com/qna/detail.naver?d1id=3&dirId=3010203&docId=436736796&qb=7ISkIOyEoOusvA==&enc=utf8&section=kin&rank=3&search_sort=0&spq=0'

In [74]:
driver.get(sub_url)
time.sleep(2)

In [39]:
#더보기 클릭 횟수 확인
answer_num=driver.find_element(By.CSS_SELECTOR,'._answerCount.num').text
answer_num

'4'

In [40]:
click_num=(int(answer_num)-1) // 5 #4개의 답변이 있으므로 한번도 안눌러도 됨
click_num

0

In [41]:
#더보기 횟수만큼 클릭 (횟수만큼 반복문)
for _ in range(click_num):
    driver.find_element(By.ID,'nextPageButton').click()
    time.sleep(2)

In [42]:
#펼쳐진 화면을 Beautifulsoup으로 파싱
sub_soup=BeautifulSoup(driver.page_source,'html.parser')
divs= sub_soup.select('.answer-content__list._answerList > div') #답변이 담길 변수
len(divs)

4

In [43]:
#답변 목록에서 개별 답변을 선택하고 글을 추출
div=divs[0] #답변 중 하나를 가져옴
ps=div.select('.se-text-paragraph.se-text-paragraph-align-')
len(ps)

4

In [44]:
ps

[<p class="se-text-paragraph se-text-paragraph-align-" id="SE-b2552918-97ba-11ed-83c9-15c95ea3c47a" style=""><span class="se-fs- se-ff-" id="SE-e5c668b0-97ba-11ed-83c9-35e1e55025bd" style="">저도 과일로 드리는데 일반딸기 말고</span></p>,
 <p class="se-text-paragraph se-text-paragraph-align-" id="SE-e5c6419a-97ba-11ed-83c9-61803cab6369" style=""><span class="se-fs- se-ff-" id="SE-e5c668b1-97ba-11ed-83c9-956919d556ee" style="">사이즈 큰 실한 딸기 드리면 아 좋은과일 사왔구나!</span></p>,
 <p class="se-text-paragraph se-text-paragraph-align-" id="SE-e5c6419c-97ba-11ed-83c9-756b52628589" style=""><span class="se-fs- se-ff-" id="SE-e5c668b2-97ba-11ed-83c9-b9466c7283b6" style="">느끼실겁니다. 딸기자체도 지금 비싼 시즌이구요.</span></p>,
 <p class="se-text-paragraph se-text-paragraph-align-" id="SE-e5c6419e-97ba-11ed-83c9-cd6d9d00b4d6" style=""><span class="se-fs- se-ff-" id="SE-e5c668b3-97ba-11ed-83c9-350f03ae9767" style="">선물하셔도 됩니다. 채택 부탁드립니다.</span></p>]

In [45]:
#리스트에서 텍스트만 뽑아오기
answer=''
for p in ps:
    text=p.get_text().strip()
    if text == '\u200b' or text=='':
        continue
    answer += text+'\n'
print(answer)    

저도 과일로 드리는데 일반딸기 말고
사이즈 큰 실한 딸기 드리면 아 좋은과일 사왔구나!
느끼실겁니다. 딸기자체도 지금 비싼 시즌이구요.
선물하셔도 됩니다. 채택 부탁드립니다.



In [46]:
#답변목록에 있는 모든 답변을 선택해서 글을 추출
answer_list=[] #4개의 답변이 텍스트화돼서 원소로 들어감
for div in divs:
    ps=div.select('.se-text-paragraph.se-text-paragraph-align-')
    answer=''
    for p in ps:
        text=p.get_text().strip()
        if text == '\u200b' or text=='':
            continue
        answer += text+'\n'
    answer_list.append(answer)    


In [47]:
answer_list

['저도 과일로 드리는데 일반딸기 말고\n사이즈 큰 실한 딸기 드리면 아 좋은과일 사왔구나!\n느끼실겁니다. 딸기자체도 지금 비싼 시즌이구요.\n선물하셔도 됩니다. 채택 부탁드립니다.\n',
 '딸기도 좋아요 근데 보관이 오래되지않아서 주는 입장도 사자마자 줘야하고 받는 분도 빨리 먹어야한다는 부담이 있을 수 있어요 냉동해먹긴 또 아깝구요ㅠㅠ 곶감정도는 어떠실까요~\n',
 '안녕하세요\n설 선물로 딸기는 괜찮아 보이구요\n아래 답변 해주신데로 알맹이가 작은 것 보단\n선물용 큼직큼직한 제품으로 포장도 잘 되어있는 제품으로\n선물하시면 좋을 것 같습니다\nhttps://link.coupang.com/a/MILwV\n"이 포스팅은 쿠팡 파트너스 활동의 일환으로, 이에 따른 일정액의 수수료를 제공받습니다."\n감사합니다.\n',
 '']

In [48]:
len(answer_list)

4

In [49]:
driver.back() #전페이지로 돌아가기

한화면의 글 10개에서 모든 답변을 추출

In [54]:
answer_list=[]
driver=webdriver.Chrome()
for page in range(1,11):
    url = f'https://kin.naver.com/search/list.naver?query={quote("설 선물")}&page={page}'
    driver=webdriver.Chrome()
    driver.get(url)
    time.sleep(2)
    soup=BeautifulSoup(driver.page_source,'html.parser')
    lis=soup.select('.basic1>li')
    
    for li in lis:
        sub_url=li.select_one('a._searchListTitleAnchor')['href'] #서브url
        driver.get(sub_url)
        time.sleep(3)
    
        answer_num=driver.find_element(By.CSS_SELECTOR,'._answerCount.num').text
        click_num=(int(answer_num)-1) // 5
        print(click_num, end=' ') #답변이 몇개인지 알 수 있음
        for _ in range(click_num):
            driver.find_element(By.ID,'nextPageButton').click()
            time.sleep(3)

        sub_soup=BeautifulSoup(driver.page_source,'html.parser')
        divs= sub_soup.select('.answer-content__list._answerList > div')
        for div in divs:
            ps=div.select('.se-text-paragraph.se-text-paragraph-align-')
            answer=''
            for p in ps:
                text=p.get_text().strip()
                if text == '\u200b' or text=='':
                    continue
                answer += text+'\n'
            answer_list.append(answer)
        driver.back()
        time.sleep(2)
    print()
driver.close()  

0 1 2 1 2 0 2 0 0 2 
1 2 5 2 0 3 0 2 1 3 
3 2 1 1 1 2 2 1 0 0 
1 0 1 0 3 0 2 0 2 0 
0 0 1 1 2 1 0 1 0 0 
0 1 0 4 0 1 0 0 0 0 
2 0 0 0 1 1 1 2 2 0 
0 2 3 0 0 0 1 2 1 0 
0 2 0 1 0 1 0 1 0 0 
0 0 1 0 1 1 0 1 0 0 


In [55]:
len(answer_list) #720개

720

파일로 저장

In [56]:
with open('설 선물.txt', 'w', encoding='utf-8') as file:
    for answer in answer_list:
        file.write(answer+'\n')